In [5]:
import warnings
warnings.filterwarnings('ignore')

import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Input, Conv2D, MaxPooling2D, UpSampling2D, Flatten
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam
from sklearn.utils import class_weight

In [6]:
# DATA DIRECTORIES
# =========================
train_dir = '../data/train'
test_dir = '../data/test'
val_dir = '../data/val'

IMG_SIZE = (224, 224)
BATCH_SIZE = 32

In [7]:
# DATA AUGMENTATION
# =========================
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='binary', shuffle=True
)

validation_generator = val_datagen.flow_from_directory(
    val_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='binary', shuffle=False
)

test_generator = test_datagen.flow_from_directory(
    test_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='binary', shuffle=False
)


Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [8]:
# CLASS WEIGHTS
# =========================
class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)
class_weights = dict(enumerate(class_weights))
print("Computed Class Weights:", class_weights)


Computed Class Weights: {0: np.float64(1.9448173005219984), 1: np.float64(0.6730322580645162)}


In [10]:
# =========================
# AUTOENCODER
# =========================
autoencoder_path = "autoencoder_model.h5"
encoder_path = "encoder_model.h5"

if os.path.exists(autoencoder_path) and os.path.exists(encoder_path):
    print("\nLoading pre-trained Autoencoder and Encoder...")
    autoencoder = load_model(autoencoder_path)
    encoder = load_model(encoder_path)
else:
    print("\nTraining Autoencoder...")

    # ⚡ Separate ImageDataGenerator with no labels
    ae_datagen = ImageDataGenerator(rescale=1./255)

    ae_train_generator = ae_datagen.flow_from_directory(
        train_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode=None,   # no labels
        shuffle=True
    )

    ae_val_generator = ae_datagen.flow_from_directory(
        val_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode=None,   # no labels
        shuffle=False
    )

    # Define Autoencoder
    input_img = Input(shape=(224, 224, 3))

    # Encoder
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    encoded = MaxPooling2D((2, 2), padding='same')(x)

    # Decoder
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(encoded)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

    autoencoder = Model(input_img, decoded)
    encoder = Model(input_img, encoded)

    autoencoder.compile(optimizer='adam', loss='mse')

    autoencoder.fit(
        ae_train_generator,
        epochs=10,
        validation_data=ae_val_generator
    )

    autoencoder.save(autoencoder_path)
    encoder.save(encoder_path)
    print("Autoencoder & Encoder Saved!")



Training Autoencoder...
Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Epoch 1/10


ValueError: None values not supported.

In [7]:
# AUTOENCODER TRAINING
# =========================
ae_train_gen_raw = ImageDataGenerator(rescale=1./255).flow_from_directory(
    train_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode=None, shuffle=True
)
ae_val_gen_raw = ImageDataGenerator(rescale=1./255).flow_from_directory(
    val_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode=None, shuffle=True
)

def autoencoder_generator(gen):
    for batch in gen:
        yield (batch, batch)

print("\nTraining Autoencoder...")
autoencoder.fit(
    autoencoder_generator(ae_train_gen_raw),
    steps_per_epoch=len(ae_train_gen_raw),
    epochs=5,
    validation_data=autoencoder_generator(ae_val_gen_raw),
    validation_steps=len(ae_val_gen_raw)
)

Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.

Training Autoencoder...
Epoch 1/5
163/163 ━━━━━━━━━━━━━━━━━━━━ 258s 2s/step - loss: 0.0074 - val_loss: 0.0020
Epoch 2/5
163/163 ━━━━━━━━━━━━━━━━━━━━ 254s 2s/step - loss: 0.0016 - val_loss: 0.0015
Epoch 3/5
163/163 ━━━━━━━━━━━━━━━━━━━━ 201s 1s/step - loss: 0.0012 - val_loss: 0.0016
Epoch 4/5
163/163 ━━━━━━━━━━━━━━━━━━━━ 178s 1s/step - loss: 0.0011 - val_loss: 0.0012
Epoch 5/5
163/163 ━━━━━━━━━━━━━━━━━━━━ 180s 1s/step - loss: 0.0010 - val_loss: 0.0011


In [8]:
# RECONSTRUCT DATASETS# RECONSTRUCT DATASETS
# =========================
def build_reconstructed_dataset(labeled_gen, model_ae):
    x_list, y_list = [], []
    labeled_gen.reset()
    for _ in range(len(labeled_gen)):
        x_batch, y_batch = next(labeled_gen)
        x_recon = model_ae.predict(x_batch, verbose=0)
        x_list.append(x_recon)
        y_list.append(y_batch)
    return np.concatenate(x_list), np.concatenate(y_list)

print("\nReconstructing datasets...")
X_train_recon, y_train = build_reconstructed_dataset(train_generator, autoencoder)
X_val_recon, y_val     = build_reconstructed_dataset(validation_generator, autoencoder)
X_test_recon, y_test   = build_reconstructed_dataset(test_generator, autoencoder)

print("Train:", X_train_recon.shape, "Val:", X_val_recon.shape, "Test:", X_test_recon.shape)# RECONSTRUCT DATASETS
# =========================
def build_reconstructed_dataset(labeled_gen, model_ae):
    x_list, y_list = [], []
    labeled_gen.reset()
    for _ in range(len(labeled_gen)):
        x_batch, y_batch = next(labeled_gen)
        x_recon = model_ae.predict(x_batch, verbose=0)
        x_list.append(x_recon)
        y_list.append(y_batch)
    return np.concatenate(x_list), np.concatenate(y_list)

print("\nReconstructing datasets...")
X_train_recon, y_train = build_reconstructed_dataset(train_generator, autoencoder)
X_val_recon, y_val     = build_reconstructed_dataset(validation_generator, autoencoder)
X_test_recon, y_test   = build_reconstructed_dataset(test_generator, autoencoder)

print("Train:", X_train_recon.shape, "Val:", X_val_recon.shape, "Test:", X_test_recon.shape)
# =========================
def build_reconstructed_dataset(labeled_gen, model_ae):
    x_list, y_list = [], []
    labeled_gen.reset()
    for _ in range(len(labeled_gen)):
        x_batch, y_batch = next(labeled_gen)
        x_recon = model_ae.predict(x_batch, verbose=0)
        x_list.append(x_recon)
        y_list.append(y_batch)
    return np.concatenate(x_list), np.concatenate(y_list)

print("\nReconstructing datasets...")
X_train_recon, y_train = build_reconstructed_dataset(train_generator, autoencoder)
X_val_recon, y_val     = build_reconstructed_dataset(validation_generator, autoencoder)
X_test_recon, y_test   = build_reconstructed_dataset(test_generator, autoencoder)

print("Train:", X_train_recon.shape, "Val:", X_val_recon.shape, "Test:", X_test_recon.shape)


Reconstructing datasets...
Train: (5216, 224, 224, 3) Val: (16, 224, 224, 3) Test: (624, 224, 224, 3)

Reconstructing datasets...
Train: (5216, 224, 224, 3) Val: (16, 224, 224, 3) Test: (624, 224, 224, 3)

Reconstructing datasets...
Train: (5216, 224, 224, 3) Val: (16, 224, 224, 3) Test: (624, 224, 224, 3)


In [10]:
# 3. CLASSIFIER ON TOP OF ENCODER
# =====================
encoder.trainable = False   # freeze encoder

x = encoder.output
x = Dense(256, activation="relu")(x)
x = Dropout(0.5)(x)
classifier_output = Dense(1, activation="sigmoid")(x)

classifier = Model(encoder.input, classifier_output, name="classifier")
classifier.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# Callbacks
def scheduler(epoch, lr):
    return lr * 0.9
lr_scheduler = LearningRateScheduler(scheduler)
early_stop = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)

print("\nTraining classifier...")
history = classifier.fit(
    train_generator,
    epochs=5,
    validation_data=validation_generator,
    callbacks=[lr_scheduler, early_stop]
)

NameError: name 'encoder' is not defined

In [9]:
# TRAIN CLASSIFIER
# =========================
lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=3, verbose=1)
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

print("\nTraining classifier...")
history = classifier.fit(
    X_train_recon, y_train,
    batch_size=BATCH_SIZE,
    epochs=10,
    validation_data=(X_val_recon, y_val),
    class_weight=class_weights,
    callbacks=[lr_scheduler, early_stop]
)


Training classifier...


NameError: name 'classifier' is not defined